In [1]:

import copy 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Datasets

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
import torchvision.models as models
pretrained_model = models.vgg11_bn(pretrained=True) 
print(pretrained_model) #pretrained_model = True -> 미리 사전에 학습된 파라미터를 사용하겠다. 

c:\Users\hop09\Desktop\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hop09\Desktop\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to C:\Users\hop09/.cache\torch\hub\checkpoints\vgg11_bn-6002323d.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

# 데이터 전처리

In [5]:
train_transforms = transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.RandomRotation(5),
                        transforms.RandomHorizontalFlip(0.5),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,  0.224, 0.225])]) 

test_transforms = transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,  0.224, 0.225])])

In [6]:
train_path = 'C:/Users/hop09/Downloads/dogs-vs-cats/train_set'
test_path = 'C:/Users/hop09/Downloads/dogs-vs-cats/test_set'


train_dataset = torchvision.datasets.ImageFolder( # 엥 이미지폴더 뭐임
    train_path,
    transform=train_transforms
) 

test_dataset = torchvision.datasets.ImageFolder(
    test_path,
    transform=test_transforms
)

print(len(train_dataset)), print(len(test_dataset))

4004
2023


(None, None)

In [26]:
VALID_RATIO = 0.9
n_train_samples = int(len(train_dataset)*VALID_RATIO )
n_valid_samples = len(train_dataset) - n_train_samples

train_data, valid_data = data.random_split(train_dataset, [n_train_samples,n_valid_samples ])

In [27]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

print("Training_n :", len(train_data))
print("valid_n :", len(valid_data))
print("test_n :", len(test_dataset))

Training_n : 3603
valid_n : 401
test_n : 2023


In [30]:
# 배치 사이즈만큼 데이터 불러오기 
BATCH_SIZE = 128
train_iterator = data.DataLoader(train_data, shuffle= True, batch_size= BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_dataset,
                                 batch_size=BATCH_SIZE)

print(len(train_iterator), len(valid_iterator), len(test_iterator))

29 4 16


# 앞서 정의한 모델의 옵티마이저, 손실함수 정하기

In [20]:
optimizer = optim.Adam(pretrained_model.parameters(), lr = 1e-7)
criterion = nn.CrossEntropyLoss()

model = pretrained_model.to(device)
criterion = criterion.to(device)

In [41]:
def calculate_accuracy(y_pred, y) :
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.size).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [38]:
def train(model, iterator, optimizer, criterion, device) :
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for (x, y) in iterator : 
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        y_pred = model(x) # y_pred, _  -> 손으로 코딩했을 때, 사전학습된 모델은 y_pred, 만 반환
        loss = criterion(y_pred, y)
        acc = calculate_accuracy(y_pred, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
        


In [39]:
def evaluate(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (x, y) in iterator:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            acc = calculate_accuracy(y_pred, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
EPOCHS = 5
best_vaild_loss = float('inf')

for epoch in range(EPOCHS) :
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model,valid_iterator, optimizer, criterion, device)
    
    if valid_loss < best_vaild_loss :
        best_vaild_loss = valid_loss
        torch.save(model.state_dict(), 'C:/Users/hop09/Downloads/dogs-vs-cats/model.pt')
        
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Valid. Loss: {valid_loss:.3f} | Valid. Acc: {valid_acc*100:.2f}%')
    

In [ ]:
def get_predictions(model, iterator):
    model.eval()
    images = []
    labels = []
    probs = []

    with torch.no_grad():
        for (x, y) in iterator:
            x = x.to(device)
            y_pred, _ = model(x)
            y_prob = F.softmax(y_pred, dim=-1)
            top_pred = y_prob.argmax(1, keepdim=True) ------ ①
            images.append(x.cpu())
            labels.append(y.cpu())
            probs.append(y_prob.cpu())

    images = torch.cat(images, dim=0) ------ ②
    labels = torch.cat(labels, dim=0)
    probs = torch.cat(probs, dim=0)
    return images, labels, probs